# Semester 1 Project Submission

Please fill out:
* Student name: Cristian González Ramírez
* Instructor name: Nick McCarty & Julian Ward

We will start by adding the basic libraries to our notebook, so it runs nice and smoothly.

In [249]:
# Import all libraries the project will requiere

#For retreiving data
import pandas as pd
import sqlite3
import zipfile

#For analyzing data
from scipy import stats

#For plotting and visulaizations
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

Then, we insert the data we will be working with.

In [250]:
#Retrieving data to work with
movie_gross_data = pd.read_csv("zippedData/bom.movie_gross.csv.gz")
movies           = pd.read_csv("zippedData/tmdb.movies.csv.gz")
movie_budgets    = pd.read_csv("zippedData/tn.movie_budgets.csv.gz")

movie_info_data  = pd.read_csv("zippedData/rt.movie_info.tsv.gz", sep="\t")

In [251]:
#Oppening SQLite3 zip
with zipfile.ZipFile("zippedData/im.db.zip", "r") as zObject:
        zObject.extractall()

conn = sqlite3.connect("im.db")
cur = conn.cursor()

Specifically, we are interested in knowing which was the gross revenue so the `movie_gross_data` table will be of major interest for us.

In [252]:
movie_gross_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


As seen in the table above, there are a couple of Null values in the `studio`, `domestic_gross` and `foreign_gross` columns, so we must clean this data out using the most convenient approach.

The `movie_gross_data.info()` command also showed us that `foreign_gross` is an object, when we need it to be a number (either an integer or a float), so we will first fix that.

In [253]:
#Getting rid off commas so it can be casted
movie_gross_data.foreign_gross.replace(",","",regex=True, inplace=True)

#Casting foreign_gross as a float
movie_gross_data["foreign_gross"] = movie_gross_data.foreign_gross.astype(float)

As we saw, many numerical values were missing, but we actually do not know wheter the film was profitable or not, and assigning any significant value to it, such as the mean of grosses or the median, could bias our analysis, so we are assigning them a `0`, in order to keep those results away from significant data.

In [254]:
#If any, filling null values with 0
movie_gross_data.foreign_gross.fillna(0, inplace=True)
movie_gross_data.domestic_gross.fillna(0, inplace=True)

In [255]:
#Getting read of NaN
movie_gross_data.dropna(inplace = True)

In [256]:
#Creating a new column for total gross
movie_gross_data["total_gross"] = movie_gross_data.foreign_gross.astype(float) +\
                                  movie_gross_data.domestic_gross.astype(float)

Now we check our results.

In [257]:
movie_gross_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3382 entries, 0 to 3386
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3382 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3382 non-null   float64
 3   foreign_gross   3382 non-null   float64
 4   year            3382 non-null   int64  
 5   total_gross     3382 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 185.0+ KB


In [258]:
movie_gross_data.head()

,title,studio,domestic_gross,foreign_gross,year,total_gross
0,Toy Story 3,BV,415000000.0,652000000.0,2010,1.067000e+09
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010,1.025500e+09
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010,9.603000e+08
3,Inception,WB,292600000.0,535700000.0,2010,8.283000e+08
4,Shrek Forever After,P/DW,238700000.0,513900000.0,2010,7.526000e+08


We can see most of it is clean and we didn't take away that many rows to make the analysis less significant.

## In the following cells I am trying to give an answer to the following question:

How are the production budgets for movies of different genres in comparison to their income? Would it be profitable?

There are a lot of values in movie_gross_data in which we are not interested, must of them being the $0 USD revenue. Those values may result as strong outliers for future analysis, so it will be convenient to manage our data without them.

In [259]:
movie_general_gross = movie_gross_data[(movie_gross_data.foreign_gross != 0) &
                                       (movie_gross_data.domestic_gross !=0)
                                      ].sort_values(by=["total_gross"], ascending=False)
movie_general_gross

,title,studio,domestic_gross,foreign_gross,year,total_gross
727,Marvel's The Avengers,BV,623400000.0,895500000.0,2012,1.518900e+09
1875,Avengers: Age of Ultron,BV,459000000.0,946400000.0,2015,1.405400e+09
3080,Black Panther,BV,700100000.0,646900000.0,2018,1.347000e+09
328,Harry Potter and the Deathly Hallows Part 2,WB,381000000.0,960500000.0,2011,1.341500e+09
2758,Star Wars: The Last Jedi,BV,620200000.0,712400000.0,2017,1.332600e+09
...,...,...,...,...,...,...
711,I'm Glad My Mother is Alive,Strand,8700.0,13200.0,2011,2.190000e+04
322,The Thorn in the Heart,Osci.,7400.0,10500.0,2010,1.790000e+04
1110,Cirkus Columbia,Strand,3500.0,9500.0,2012,1.300000e+04
715,Aurora,CGld,5700.0,5100.0,2011,1.080000e+04


In [260]:
#Obtain relevant information from SQL database
query = '''
    SELECT
        movie_id AS "ID",
        primary_title AS "Title",
        original_title AS "Original Title",
        genres AS "Genres",
        averagerating AS "Rating",
        numvotes AS "Number of Votes"
    FROM movie_basics
        JOIN movie_ratings
        USING (movie_id)
'''

#Assign the query to a pandas dataframe
rating_by_title = pd.read_sql(query,conn)

In [261]:
rating_by_title.head(10)

,ID,Title,Original Title,Genres,Rating,Number of Votes
0,tt0063540,Sunghursh,Sunghursh,"Action,Crime,Drama",7.0,77
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,"Biography,Drama",7.2,43
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,Drama,6.9,4517
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,"Comedy,Drama",6.1,13
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,"Comedy,Drama,Fantasy",6.5,119
5,tt0112502,Bigfoot,Bigfoot,"Horror,Thriller",4.1,32
6,tt0137204,Joe Finds Grace,Joe Finds Grace,"Adventure,Animation,Comedy",8.1,263
7,tt0146592,Pál Adrienn,Pál Adrienn,Drama,6.8,451
8,tt0154039,So Much for Justice!,Oda az igazság,History,4.6,64
9,tt0159369,Cooper and Hemingway: The True Gen,Cooper and Hemingway: The True Gen,Documentary,7.6,53


In [262]:
rating_by_title.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               73856 non-null  object 
 1   Title            73856 non-null  object 
 2   Original Title   73856 non-null  object 
 3   Genres           73052 non-null  object 
 4   Rating           73856 non-null  float64
 5   Number of Votes  73856 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 3.4+ MB


As we can see, most of the data we obtain is complete, but there may still be some values that could mess up our analysis, so we better clean this data as well!

In [263]:
rating_by_title.Title.value_counts()

The Return              11
Broken                  10
Together                 9
Lucky                    9
Homecoming               9
                        ..
Peculiar Pets            1
Pour l'amour de Dieu     1
After Party              1
Red Cow                  1
Shilo                    1
Name: Title, Length: 69993, dtype: int64

In [264]:
rating_by_title.drop_duplicates(subset=["Title"], inplace = True)

In [265]:
rating_by_title.Title.value_counts()

Violentia                 1
Searching                 1
Money and Medicine        1
The Funhouse Massacre     1
Senza lasciare traccia    1
                         ..
These Birds Walk          1
Trumpocalypse Now!        1
Night School              1
Deadfall                  1
Shilo                     1
Name: Title, Length: 69993, dtype: int64

In [266]:
rating_by_title["Original Title"].value_counts()

Missed Connections          3
Mundo Cão                   2
Iris                        2
Ra                          2
Snatched                    2
                           ..
Auf den zweiten Blick       1
Angelus Hiroshimae          1
Molly's Girl                1
The Melancholy Fantastic    1
Shilo                       1
Name: Original Title, Length: 69820, dtype: int64

In [267]:
rating_by_title.drop_duplicates(subset=["Original Title"], inplace = True)

In [268]:
rating_by_title["Original Title"].value_counts()

Violentia                       1
C'est tout pour moi             1
And Who Taught You to Drive?    1
Dhairyam                        1
Satri lek tob lok taek          1
                               ..
Qian Xuesen                     1
Die Präsenz                     1
Life of a King                  1
El Teniente Amado               1
Shilo                           1
Name: Original Title, Length: 69820, dtype: int64

In [269]:
rating_by_title.Rating.unique()

array([ 7. ,  7.2,  6.9,  6.1,  6.5,  4.1,  8.1,  6.8,  4.6,  7.6,  7.5,
        7.8,  4. ,  8.8,  1.9,  6.7,  6.6,  5.9,  7.1,  7.4,  3.9,  5.5,
        5.1,  7.9,  6.2,  6.3,  5. ,  6. ,  5.6,  7.3,  4.9,  6.4,  3.3,
        5.7,  8.6,  4.8,  4.5,  4.2,  8.7,  4.7,  5.2,  8.3,  9.1,  3.4,
        5.8,  5.4,  8.5,  3.2,  3.6,  8. ,  3.8,  7.7,  4.3,  3. ,  5.3,
        3.7,  3.5,  9.3,  8.9,  9. ,  2.3,  8.2,  2.7,  2.8,  1.7,  4.4,
        2.2,  9.4,  8.4,  9.2,  9.7,  3.1, 10. ,  9.8,  2.6,  2.5,  1.4,
        2.1,  2.4,  2.9,  1.8,  1.6,  2. ,  1. ,  1.2,  1.5,  1.3,  1.1,
        9.5,  9.6,  9.9])

We can see there is no apparent problem in the Rating column

In [270]:
rating_by_title.ID.value_counts().sort_values(ascending = True)

tt8428316    1
tt2354215    1
tt7054868    1
tt1334455    1
tt7397958    1
            ..
tt1795046    1
tt6096958    1
tt6202264    1
tt8998610    1
tt1619814    1
Name: ID, Length: 69820, dtype: int64

Nor there is any in the ID column. 

Most of the cleaning of this Dataframe is done. The next step to do is to decide how to procede with the missing values we saw in the Genres column. There are around 800 missing values, and dropping those columns won't represent as much of a loss, so we will proceed with this approach.

In [271]:
rating_by_title = rating_by_title[rating_by_title.Genres.str.contains("None") == False]

In [272]:
rating_by_title.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69059 entries, 0 to 73852
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               69059 non-null  object 
 1   Title            69059 non-null  object 
 2   Original Title   69059 non-null  object 
 3   Genres           69059 non-null  object 
 4   Rating           69059 non-null  float64
 5   Number of Votes  69059 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 3.7+ MB


We can see we have a dataframe without any null value or duplicates. We can consider this dataframe to be clean!

Nevertheless, before moving forward, I will consider an extra step for singificance in later analysis; which is getting the percentage of rating base on the number of votes. Where we will use the greatest `Number of votes` as reference to all the movies, i.e. the movie with its `Number of voters` value equal to `max_voters` will have a significance of `1`.

In [273]:
max_voters = rating_by_title["Number of Votes"].max()
rating_by_title["Significance"] = (rating_by_title["Number of Votes"]/max_voters)

In [274]:
rating_by_title.head()

,ID,Title,Original Title,Genres,Rating,Number of Votes,Significance
0,tt0063540,Sunghursh,Sunghursh,"Action,Crime,Drama",7.0,77,0.000042
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,"Biography,Drama",7.2,43,0.000023
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,Drama,6.9,4517,0.002453
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,"Comedy,Drama",6.1,13,0.000007
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,"Comedy,Drama,Fantasy",6.5,119,0.000065


In [275]:
rating_by_title.loc[(rating_by_title["Number of Votes"]) == max_voters]

,ID,Title,Original Title,Genres,Rating,Number of Votes,Significance
2387,tt1375666,Inception,Inception,"Action,Adventure,Sci-Fi",8.8,1841066,1.0


Now, we shall proceed with joining the `rating_by_title` table with the `movie_general_gross` one to proceed with the analysis.

In [276]:
#Create a new Dataframe
#gross_and_rating = 